In [12]:
import pandas as pd
import numpy as np
from binance.client import Client
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC  
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle
import os

In [2]:
def get_historical_data():
    api_key = 'iyJXPaZztWrimkH6V57RGvStFgYQWRaaMdaYBQHHIEv0mMY1huCmrzTbXkaBjLFh'

    api_secret = 'hmrus7zI9PW2EXqsDVovoS2cEFRVsxeETGgBf4XJInOLFcmIXKNL23alGRNRbXKI'
    client = Client(api_key, api_secret)
    interval = '15m'
    symbol = 'ETHUSDT'
    n = 200
    klines = client.futures_klines(symbol=symbol, interval=interval, limit=n)

    timestamps = [int(kline[0]) for kline in klines]
    open_prices = [float(kline[1]) for kline in klines]
    high_prices = [float(kline[2]) for kline in klines]
    low_prices = [float(kline[3]) for kline in klines]
    close_prices = [float(kline[4]) for kline in klines]
    date = pd.to_datetime(timestamps, unit='ms')

    # Create a DataFrame with DatetimeIndex
    df = pd.DataFrame({
        'date': date,
        'open': open_prices,
        'high': high_prices,
        'low': low_prices,
        'close': close_prices,
    }, index=pd.to_datetime(timestamps, unit='ms'))
    return df

In [4]:
df = get_historical_data()
df

,date,open,high,low,close
2023-12-14 04:15:00,2023-12-14 04:15:00,2253.57,2254.06,2251.24,2252.24
2023-12-14 04:30:00,2023-12-14 04:30:00,2252.24,2252.95,2248.14,2251.85
2023-12-14 04:45:00,2023-12-14 04:45:00,2251.85,2251.86,2246.89,2248.03
2023-12-14 05:00:00,2023-12-14 05:00:00,2248.04,2257.11,2247.56,2256.66
2023-12-14 05:15:00,2023-12-14 05:15:00,2256.66,2262.38,2255.48,2260.63
...,...,...,...,...,...
2023-12-16 05:00:00,2023-12-16 05:00:00,2251.29,2253.30,2249.12,2249.40
2023-12-16 05:15:00,2023-12-16 05:15:00,2249.39,2251.47,2248.20,2251.46
2023-12-16 05:30:00,2023-12-16 05:30:00,2251.45,2251.55,2246.59,2247.29
2023-12-16 05:45:00,2023-12-16 05:45:00,2247.30,2250.15,2246.74,2249.14


In [10]:
def train_base_model():
    df = get_historical_data()
    # Assuming df is your DataFrame
    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)

    # Feature Scaling
    scaler_filename = f'./minmax_scaler.pkl'
    try:
        # Load existing scaler
        with open(scaler_filename, 'rb') as scaler_file:
            scaler = pickle.load(scaler_file)
    except FileNotFoundError:
        # Create a new scaler if not found
        scaler = MinMaxScaler()
    df[['open', 'high', 'low', 'close']] = scaler.fit_transform(df[['open', 'high', 'low', 'close']])

    # Train-Test Split
    X = df[['open', 'high', 'low', 'close']]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build the model
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(64, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(32, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # You can adjust the learning rate
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=900, batch_size=200, validation_data=(X_test, y_test))

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Accuracy:", accuracy)
    with open(scaler_filename, 'wb') as scaler_file:
        pickle.dump(scaler, scaler_file)
    model.save(f'./trade_model_1min.h5')




In [11]:
train_base_model()

Epoch 1/900


C:\Users\OMEN\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.3.0 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 [==============================] - 1s 1s/step - loss: 0.7344 - accuracy: 0.5375 - val_loss: 0.6946 - val_accuracy: 0.4500
Epoch 2/900
1/1 [==============================] - 0s 22ms/step - loss: 0.6536 - accuracy: 0.6062 - val_loss: 0.6935 - val_accuracy: 0.4500
Epoch 3/900
1/1 [==============================] - 0s 19ms/step - loss: 0.6169 - accuracy: 0.6875 - val_loss: 0.6925 - val_accuracy: 0.5500
Epoch 4/900
1/1 [==============================] - 0s 20ms/step - loss: 0.6022 - accuracy: 0.6875 - val_loss: 0.6917 - val_accuracy: 0.5500
Epoch 5/900
1/1 [==============================] - 0s 19ms/step - loss: 0.5940 - accuracy: 0.7063 - val_loss: 0.6909 - val_accuracy: 0.5500
Epoch 6/900
1/1 [==============================] - 0s 19ms/step - loss: 0.5875 - accuracy: 0.7000 - val_loss: 0.6902 - val_accuracy: 0.5500
Epoch 7/900
1/1 [==============================] - 0s 19ms/step - loss: 0.5813 - accuracy: 0.7125 - val_loss: 0.6897 - val_accuracy: 0.5500
Epoch 8/900
1/1 [=================

1/1 [==============================] - 0s 19ms/step - loss: 0.3850 - accuracy: 0.8562 - val_loss: 0.6993 - val_accuracy: 0.4500
Epoch 60/900
1/1 [==============================] - 0s 18ms/step - loss: 0.3865 - accuracy: 0.8687 - val_loss: 0.6995 - val_accuracy: 0.4500
Epoch 61/900
1/1 [==============================] - 0s 18ms/step - loss: 0.3839 - accuracy: 0.8375 - val_loss: 0.7003 - val_accuracy: 0.4500
Epoch 62/900
1/1 [==============================] - 0s 18ms/step - loss: 0.3844 - accuracy: 0.8313 - val_loss: 0.7006 - val_accuracy: 0.4500
Epoch 63/900
1/1 [==============================] - 0s 18ms/step - loss: 0.3884 - accuracy: 0.8500 - val_loss: 0.7010 - val_accuracy: 0.4500
Epoch 64/900
1/1 [==============================] - 0s 18ms/step - loss: 0.3792 - accuracy: 0.8562 - val_loss: 0.7009 - val_accuracy: 0.4500
Epoch 65/900
1/1 [==============================] - 0s 19ms/step - loss: 0.3835 - accuracy: 0.8313 - val_loss: 0.7011 - val_accuracy: 0.4500
Epoch 66/900
1/1 [========

1/1 [==============================] - 0s 19ms/step - loss: 0.2733 - accuracy: 0.9125 - val_loss: 0.6942 - val_accuracy: 0.5000
Epoch 118/900
1/1 [==============================] - 0s 17ms/step - loss: 0.2719 - accuracy: 0.9125 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 119/900
1/1 [==============================] - 0s 18ms/step - loss: 0.2717 - accuracy: 0.9000 - val_loss: 0.6937 - val_accuracy: 0.5250
Epoch 120/900
1/1 [==============================] - 0s 18ms/step - loss: 0.2696 - accuracy: 0.9000 - val_loss: 0.6936 - val_accuracy: 0.5250
Epoch 121/900
1/1 [==============================] - 0s 19ms/step - loss: 0.2662 - accuracy: 0.9125 - val_loss: 0.6936 - val_accuracy: 0.5250
Epoch 122/900
1/1 [==============================] - 0s 18ms/step - loss: 0.2576 - accuracy: 0.9187 - val_loss: 0.6935 - val_accuracy: 0.4750
Epoch 123/900
1/1 [==============================] - 0s 19ms/step - loss: 0.2638 - accuracy: 0.9125 - val_loss: 0.6932 - val_accuracy: 0.4500
Epoch 124/900
1/1 [=

...

Epoch 639/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0237 - accuracy: 0.9937 - val_loss: 2.2333 - val_accuracy: 0.5250
Epoch 640/900
1/1 [==============================] - 0s 20ms/step - loss: 0.0232 - accuracy: 0.9937 - val_loss: 2.0941 - val_accuracy: 0.5000
Epoch 641/900
1/1 [==============================] - 0s 20ms/step - loss: 0.0239 - accuracy: 1.0000 - val_loss: 2.1751 - val_accuracy: 0.5000
Epoch 642/900
1/1 [==============================] - 0s 20ms/step - loss: 0.0218 - accuracy: 1.0000 - val_loss: 2.1806 - val_accuracy: 0.4750
Epoch 643/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0228 - accuracy: 1.0000 - val_loss: 2.1332 - val_accuracy: 0.4750
Epoch 644/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0231 - accuracy: 1.0000 - val_loss: 2.2567 - val_accuracy: 0.4750
Epoch 645/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0217 - accuracy: 1.0000 - val_loss: 2.3650 - val_accuracy: 0.4750
Epoch 

Epoch 697/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0187 - accuracy: 0.9937 - val_loss: 2.0292 - val_accuracy: 0.5000
Epoch 698/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0181 - accuracy: 1.0000 - val_loss: 1.9657 - val_accuracy: 0.4750
Epoch 699/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0189 - accuracy: 1.0000 - val_loss: 2.0752 - val_accuracy: 0.4750
Epoch 700/900
1/1 [==============================] - 0s 20ms/step - loss: 0.0187 - accuracy: 0.9937 - val_loss: 1.9577 - val_accuracy: 0.5000
Epoch 701/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0176 - accuracy: 0.9937 - val_loss: 2.1251 - val_accuracy: 0.5250
Epoch 702/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0173 - accuracy: 1.0000 - val_loss: 2.1186 - val_accuracy: 0.4750
Epoch 703/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0171 - accuracy: 1.0000 - val_loss: 2.0573 - val_accuracy: 0.4750
Epoch 

Epoch 755/900
1/1 [==============================] - 0s 17ms/step - loss: 0.0119 - accuracy: 1.0000 - val_loss: 2.7388 - val_accuracy: 0.3750
Epoch 756/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 2.8781 - val_accuracy: 0.3750
Epoch 757/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0127 - accuracy: 1.0000 - val_loss: 2.8990 - val_accuracy: 0.4250
Epoch 758/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0122 - accuracy: 1.0000 - val_loss: 2.8064 - val_accuracy: 0.4000
Epoch 759/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0133 - accuracy: 1.0000 - val_loss: 2.9183 - val_accuracy: 0.4500
Epoch 760/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0149 - accuracy: 1.0000 - val_loss: 2.7146 - val_accuracy: 0.3750
Epoch 761/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0152 - accuracy: 0.9937 - val_loss: 3.0304 - val_accuracy: 0.4250
Epoch 

Epoch 813/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0105 - accuracy: 1.0000 - val_loss: 2.9219 - val_accuracy: 0.3750
Epoch 814/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 2.8450 - val_accuracy: 0.4250
Epoch 815/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0129 - accuracy: 1.0000 - val_loss: 2.7036 - val_accuracy: 0.4000
Epoch 816/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0111 - accuracy: 1.0000 - val_loss: 2.6769 - val_accuracy: 0.3750
Epoch 817/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 2.7541 - val_accuracy: 0.4250
Epoch 818/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 2.7899 - val_accuracy: 0.3750
Epoch 819/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 2.6769 - val_accuracy: 0.4000
Epoch 

Epoch 871/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0101 - accuracy: 1.0000 - val_loss: 3.5383 - val_accuracy: 0.3750
Epoch 872/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 3.3546 - val_accuracy: 0.3500
Epoch 873/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 3.4386 - val_accuracy: 0.3500
Epoch 874/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 3.3201 - val_accuracy: 0.3500
Epoch 875/900
1/1 [==============================] - 0s 18ms/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 3.3531 - val_accuracy: 0.3500
Epoch 876/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0087 - accuracy: 1.0000 - val_loss: 3.3886 - val_accuracy: 0.3250
Epoch 877/900
1/1 [==============================] - 0s 19ms/step - loss: 0.0079 - accuracy: 1.0000 - val_loss: 3.3315 - val_accuracy: 0.3000
Epoch 

In [22]:
def predict_crypto():
    """

    Returns: predicted dataframe

    """
    new_data = get_historical_data()
    loaded_model = tf.keras.models.load_model(f'./trade_model_1min.h5')
    scaler_filename = f'./minmax_scaler.pkl'

    try:
        # Load existing scaler
        with open(scaler_filename, 'rb') as scaler_file:
            scaler = pickle.load(scaler_file)
    except FileNotFoundError:
        # Create a new scaler if not found
        scaler = MinMaxScaler()
    new_data[['open', 'high', 'low', 'close']] = scaler.transform(new_data[['open', 'high', 'low', 'close']])
    X_new = new_data[['open', 'high', 'low', 'close']]
    y_new = (new_data['close'].shift(-1) > new_data['close']).astype(int)
    X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.2, random_state=42)
    loaded_model.fit(X_train_new, y_train_new, epochs=900, batch_size=10, validation_data=(X_test_new, y_test_new))
    new_data['predicted_prob'] = loaded_model.predict(X_new)
    threshold = 0.5
    new_data['trading_signal'] = np.where(new_data['predicted_prob'] > threshold, 1, -1)
    print(new_data[['close', 'predicted_prob', 'trading_signal']].iloc[-1])
    loaded_model.save('.trade_model_1min.h5')
    return new_data[['close', 'predicted_prob', 'trading_signal']].iloc[-1]



In [23]:
predict_crypto()

Epoch 1/900
16/16 [==============================] - 1s 10ms/step - loss: 1.5677 - accuracy: 0.5500 - val_loss: 21.3535 - val_accuracy: 0.5750
Epoch 2/900
16/16 [==============================] - 0s 2ms/step - loss: 1.0074 - accuracy: 0.5813 - val_loss: 2.6513 - val_accuracy: 0.4750
Epoch 3/900
16/16 [==============================] - 0s 2ms/step - loss: 0.7813 - accuracy: 0.5625 - val_loss: 1.8340 - val_accuracy: 0.4750
Epoch 4/900
16/16 [==============================] - 0s 2ms/step - loss: 0.7100 - accuracy: 0.5875 - val_loss: 1.8945 - val_accuracy: 0.5000
Epoch 5/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6639 - accuracy: 0.5750 - val_loss: 1.7255 - val_accuracy: 0.4500
Epoch 6/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6502 - accuracy: 0.6125 - val_loss: 1.2896 - val_accuracy: 0.5250
Epoch 7/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6565 - accuracy: 0.5750 - val_loss: 1.0226 - val_accuracy: 0.4750
Epoch 8/900

Epoch 59/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6344 - accuracy: 0.6438 - val_loss: 0.9359 - val_accuracy: 0.4250
Epoch 60/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6303 - accuracy: 0.6250 - val_loss: 0.8142 - val_accuracy: 0.4500
Epoch 61/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6002 - accuracy: 0.6812 - val_loss: 0.8223 - val_accuracy: 0.4250
Epoch 62/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6643 - accuracy: 0.6438 - val_loss: 0.8345 - val_accuracy: 0.3750
Epoch 63/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5907 - accuracy: 0.6750 - val_loss: 0.8244 - val_accuracy: 0.3750
Epoch 64/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6818 - accuracy: 0.5875 - val_loss: 0.7933 - val_accuracy: 0.4250
Epoch 65/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6293 - accuracy: 0.6500 - val_loss: 0.7988 - val_accuracy: 0.4000
Epoch 

16/16 [==============================] - 0s 2ms/step - loss: 0.6352 - accuracy: 0.6687 - val_loss: 0.8564 - val_accuracy: 0.3750
Epoch 117/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6438 - accuracy: 0.6187 - val_loss: 0.8021 - val_accuracy: 0.4500
Epoch 118/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6199 - accuracy: 0.6062 - val_loss: 0.8040 - val_accuracy: 0.4000
Epoch 119/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6562 - accuracy: 0.6062 - val_loss: 0.8058 - val_accuracy: 0.4000
Epoch 120/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6291 - accuracy: 0.6562 - val_loss: 0.8158 - val_accuracy: 0.4250
Epoch 121/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6092 - accuracy: 0.7375 - val_loss: 0.9434 - val_accuracy: 0.4500
Epoch 122/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6047 - accuracy: 0.6625 - val_loss: 0.8444 - val_accuracy: 0.4000
Epoch 123/900

16/16 [==============================] - 0s 2ms/step - loss: 0.6022 - accuracy: 0.6938 - val_loss: 0.8123 - val_accuracy: 0.4000
Epoch 174/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6319 - accuracy: 0.6250 - val_loss: 0.8488 - val_accuracy: 0.4250
Epoch 175/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6344 - accuracy: 0.6375 - val_loss: 0.7917 - val_accuracy: 0.4250
Epoch 176/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6086 - accuracy: 0.6250 - val_loss: 0.7547 - val_accuracy: 0.4250
Epoch 177/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6223 - accuracy: 0.6875 - val_loss: 0.7555 - val_accuracy: 0.4750
Epoch 178/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6502 - accuracy: 0.6438 - val_loss: 0.9460 - val_accuracy: 0.4500
Epoch 179/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6212 - accuracy: 0.6000 - val_loss: 0.8462 - val_accuracy: 0.4500
Epoch 180/900

16/16 [==============================] - 0s 2ms/step - loss: 0.6207 - accuracy: 0.6500 - val_loss: 0.8260 - val_accuracy: 0.4000
Epoch 231/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6097 - accuracy: 0.6562 - val_loss: 0.8197 - val_accuracy: 0.4500
Epoch 232/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5912 - accuracy: 0.7063 - val_loss: 0.8306 - val_accuracy: 0.4500
Epoch 233/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6077 - accuracy: 0.6750 - val_loss: 0.8177 - val_accuracy: 0.4000
Epoch 234/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5845 - accuracy: 0.6875 - val_loss: 0.8705 - val_accuracy: 0.4250
Epoch 235/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6295 - accuracy: 0.6375 - val_loss: 0.8858 - val_accuracy: 0.4750
Epoch 236/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5866 - accuracy: 0.6313 - val_loss: 0.8462 - val_accuracy: 0.4250
Epoch 237/900

...

16/16 [==============================] - 0s 2ms/step - loss: 0.5783 - accuracy: 0.6625 - val_loss: 1.0523 - val_accuracy: 0.4500
Epoch 687/900
16/16 [==============================] - 0s 2ms/step - loss: 0.6022 - accuracy: 0.6812 - val_loss: 1.0434 - val_accuracy: 0.4500
Epoch 688/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5547 - accuracy: 0.6938 - val_loss: 1.0200 - val_accuracy: 0.3500
Epoch 689/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5510 - accuracy: 0.7063 - val_loss: 1.1358 - val_accuracy: 0.4250
Epoch 690/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5239 - accuracy: 0.7312 - val_loss: 1.4910 - val_accuracy: 0.4500
Epoch 691/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5636 - accuracy: 0.7500 - val_loss: 1.5966 - val_accuracy: 0.4750
Epoch 692/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5541 - accuracy: 0.7250 - val_loss: 1.1705 - val_accuracy: 0.4750
Epoch 693/900

16/16 [==============================] - 0s 2ms/step - loss: 0.6291 - accuracy: 0.6438 - val_loss: 1.0101 - val_accuracy: 0.3750
Epoch 744/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5791 - accuracy: 0.6938 - val_loss: 1.0296 - val_accuracy: 0.3000
Epoch 745/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5672 - accuracy: 0.6125 - val_loss: 1.0381 - val_accuracy: 0.3250
Epoch 746/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5500 - accuracy: 0.7000 - val_loss: 1.2163 - val_accuracy: 0.4000
Epoch 747/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5692 - accuracy: 0.7125 - val_loss: 1.0692 - val_accuracy: 0.4250
Epoch 748/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5466 - accuracy: 0.7250 - val_loss: 0.9890 - val_accuracy: 0.3750
Epoch 749/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5317 - accuracy: 0.7000 - val_loss: 0.9669 - val_accuracy: 0.3750
Epoch 750/900

16/16 [==============================] - 0s 2ms/step - loss: 0.5484 - accuracy: 0.7000 - val_loss: 1.1536 - val_accuracy: 0.4250
Epoch 801/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5795 - accuracy: 0.7063 - val_loss: 1.0398 - val_accuracy: 0.3500
Epoch 802/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5582 - accuracy: 0.7063 - val_loss: 1.0192 - val_accuracy: 0.4250
Epoch 803/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5813 - accuracy: 0.6562 - val_loss: 0.9895 - val_accuracy: 0.3250
Epoch 804/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5548 - accuracy: 0.7063 - val_loss: 0.9607 - val_accuracy: 0.4250
Epoch 805/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5874 - accuracy: 0.6875 - val_loss: 1.1266 - val_accuracy: 0.3000
Epoch 806/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5238 - accuracy: 0.7312 - val_loss: 1.1133 - val_accuracy: 0.4000
Epoch 807/900

16/16 [==============================] - 0s 2ms/step - loss: 0.5169 - accuracy: 0.7500 - val_loss: 1.2542 - val_accuracy: 0.4000
Epoch 858/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5355 - accuracy: 0.7312 - val_loss: 1.2065 - val_accuracy: 0.4250
Epoch 859/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5347 - accuracy: 0.7250 - val_loss: 1.1736 - val_accuracy: 0.4500
Epoch 860/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5508 - accuracy: 0.6875 - val_loss: 1.1247 - val_accuracy: 0.3750
Epoch 861/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5919 - accuracy: 0.6750 - val_loss: 1.0526 - val_accuracy: 0.2750
Epoch 862/900
16/16 [==============================] - 0s 2ms/step - loss: 0.5452 - accuracy: 0.7125 - val_loss: 1.1506 - val_accuracy: 0.3750
Epoch 863/900
16/16 [==============================] - 0s 2ms/step - loss: 0.4994 - accuracy: 0.7625 - val_loss: 1.1253 - val_accuracy: 0.3500
Epoch 864/900

close             0.252985
predicted_prob    0.540673
trading_signal    1.000000
Name: 2023-12-16 06:15:00, dtype: float64